In [2]:
import taichi as ti

ti.init(arch=ti.gpu)

def read_obj(path):
    f = open(path)

    v_num = 0
    f_num = 0

    while f.readable():
        s = f.readline()
        if "Vertices:" in s:
            v_num = int(s.split()[-1])
        if "Faces:" in s:
            f_num = int(s.split()[-1])
        if "v" in s:
            break

    x_r = ti.Vector.field(3, dtype=float, shape=(f_num,3))
    x_rp = ti.Vector.field(3, dtype=float, shape=f_num)

    points = []
    faces = []

    for i in range(v_num):
        ps = [float(j) for j in  s.split()[1:]]
        points.append(ps)
        s = f.readline()

    while "f" not in s:
        s = f.readline()

    for i in range(f_num):
        fs = [int(j) for j in  s.split()[1:]]
        faces.append(fs)
        s = f.readline()

    for i in range(f_num):
        ps = [points[j-1] for j in faces[i]]
        for j in range(3):
            x_r[i,j] = ps[j]
        x_rp[i] = (x_r[i,0] + x_r[i,1] + x_r[i,2]) / 3
    f.close()
    return x_rp, x_r

x_rp, x_r = read_obj("./python/ring1.obj")

# x_rp = ti.Vector.field(3, dtype=float, shape = (2))
# x_r = ti.Vector.field(3, dtype=float, shape = (2,3))
# x_pp = ti.Vector.field(3, dtype=float, shape=(2))


[Taichi] version 0.8.8, llvm 10.0.0, commit 7bae9c77, win, python 3.8.12
[Taichi] Starting on arch=cuda


In [36]:
import os

dir_num = -1
for root,dirs,files in os.walk('./tmp'):
    # print(dirs)
    for temp_dir in dirs:
        dir_num = max(dir_num,int(temp_dir[3:]))
        print(dir_num)
dir_num += 1
dir_num


0
1


2

In [20]:
int(dirs[0][3:])

0

In [29]:
offset = ti.Vector([1,2,3])
if offset[0] >= 32 or offset[1] >= 21:
    print("nmsl")
else:
    print("woc")

woc


In [3]:
x_r[0,0] = [1,0,0]
x_r[0,1] = [0,1,0]
x_r[0,2] = [0,0,1]
x_rp[0] = [1/3, 1/3, 1/3]


In [21]:
i = 0
nor = (x_r[i,1]-x_r[i,0]).cross(x_r[i,2]-x_r[i,1])
d = -1*nor.dot(x_r[i,0])
print(x_r[i,1].dot(nor) + d,(x_r[i,1]+x_r[i,0]+x_r[i,2]).dot(nor) )

0.0 7.782926835241133e-06


In [5]:
normal = (x_r[0,1]-x_r[0,0]).cross(x_r[0,2]-x_r[0,1])
d = -1*normal.dot(x_r[0,0])
print(x_r[0,0].dot(normal) + d)
print(x_r[0,1].dot(normal) + d)
print(x_r[0,2].dot(normal) + d)
print(x_rp[0].dot(normal) + d)

0.0
0.0
0.0
2.9802322387695312e-08


In [23]:
48/64

0.75

In [7]:
# @ti.func
def isinplane(points, i):
    flag = False
    T = -1
    normal = (x_r[i,1]-x_r[i,0]).cross(x_r[i,2]-x_r[i,1])
    d = -1*normal.dot(x_r[i,0])
    xp = ((normal[1]**2 + normal[2]**2)*points[0] - normal[0]*(normal[1]*points[1]+normal[2]*points[2]+d)) / (normal[0]**2+normal[1]**2+normal[2]**2)
    yp = ((normal[2]**2 + normal[0]**2)*points[1] - normal[1]*(normal[0]*points[0]+normal[2]*points[2]+d)) / (normal[0]**2+normal[1]**2+normal[2]**2)
    zp = ((normal[0]**2 + normal[1]**2)*points[2] - normal[2]*(normal[0]*points[0]+normal[1]*points[1]+d)) / (normal[0]**2+normal[1]**2+normal[2]**2)
    p_pro = ti.Vector([xp, yp, zp])
    b1 = (p_pro-x_r[i,0]).cross(x_r[i,1]-x_r[i,0]).dot((x_r[i,2]-x_r[i,0]).cross(x_r[i,1]-x_r[i,0]))
    b2 = (p_pro-x_r[i,1]).cross(x_r[i,2]-x_r[i,1]).dot((x_r[i,0]-x_r[i,1]).cross(x_r[i,2]-x_r[i,1]))
    b3 = (p_pro-x_r[i,2]).cross(x_r[i,0]-x_r[i,2]).dot((x_r[i,1]-x_r[i,2]).cross(x_r[i,0]-x_r[i,2]))
    distance = abs(points.dot(normal)+d) / (normal[0]**2+normal[1]**2+normal[2]**2)
    if (b1>=0 and b2>=0 and b3>=0) or(b1<=0 and b2<=0 and b3<=0):
        flag = True
    if (points-x_rp[i]).dot(normal) > 0:
        T = 1
    return flag, distance, T, normal

In [8]:
temp_p = ti.Vector([2,0,0])
isinplane(temp_p, 0)

(False, 15607.284351178852, -1, [1.24847964e-06 2.16304178e-06 2.16783036e-10])

In [10]:
x1 = [0.521690, 0.898268, -0.007316]
x2 = [0.519384, 0.899599, -0.007785]
x3 = [0.517078, 0.900930, -0.007316]

In [13]:
for i in range(3):
    print(x3[i]-x2[i])

-0.0023059999999999192
0.001330999999999971
0.00046900000000000067
